In [1]:
# this is needed to import lumiacc 
import sys
sys.path.append('/afs/cern.ch/work/a/adlintul/public/scouting/hbb/analysis/Run3ScoutingHbb')
from processors import lumiacc

import numpy as np
import pandas as pd
import hist
from hist import Hist
import awkward as ak
import json
import uproot
import math
from coffea import util
import pandas as pd
import pickle
import os

import matplotlib.pyplot as plt
import mplhep
plt.style.use(mplhep.style.CMS)

# Table of contents <a class="anchor" id="toc"></a>

* [Scaling MC by cross section and luminosity](#scaling)
* [Grouping data together](#grouping)

# Scaling MC by cross section and luminosity <a class="anchor" id="scaling"></a>
[Back to Table of Contents](#toc)

In [2]:
# Insert directory from which the MC samples are read. Here I am reading the samples
# from the "hist" step (i.e. creation of the final histogram).
step = "hist"
out_dir = f"../outfiles/Run3Summer22EE/{step}"

# Insert lumi in units of /pb
total_lumi = 100 * 1000

# Read cross section of each dataset
with open("../data/xsec/xsec.json", "r") as json_file:
    xs = json.load(json_file)

In [3]:
# This cell scales the MC histograms by cross section and luminosity.
# It also groups various related datasets into a single processes, e.g.
# ['TTto2L2Nu', 'TTtoLNu2Q', 'TTto4Q] => TTbar

from collections import defaultdict 
import os
import pickle

outsum = defaultdict()

started = 0

for file in [
    "dask_QCD.coffea",
    "dask_Zto2Q-4Jets.coffea",
    "dask_Wto2Q-3Jets.coffea",
    "dask_TT.coffea",
    "dask_VV.coffea",
    "dask_Hto2B.coffea",
]:
    
    filename = f"{out_dir}/{file}"

    print("Loading "+filename)

    if os.path.isfile(filename):
        out = util.load(filename)[0]

        if started == 0:
            outsum['templates'] = out['hist']
            outsum['sumw'] = out['sumw']
            started += 1
        else:
            outsum['templates'] += out['hist']
            for k,v in out['sumw'].items():
                outsum['sumw'][k] = v
        del out
    else:
        print(filename + " does not exist")
                
# "dataset" has to be first for the scaling to work
outsum["templates"] = outsum["templates"].project('dataset', 'mass', 'pt', 'disc', 'genflav', 'systematic')

scale_lumi = {k: xs[k] * total_lumi / w for k, w in outsum['sumw'].items()}

for i, name in enumerate(outsum["templates"].axes["dataset"]):
    outsum["templates"].view(flow=True)[i] *= scale_lumi[name]

def group(h: hist.Hist, oldname: str, newname: str, grouping: dict):
    hnew = hist.Hist(
        hist.axis.StrCategory(grouping, name=newname),
        *(ax for ax in h.axes if ax.name != oldname),
        storage=h._storage_type,
    )
    for i, indices in enumerate(grouping.values()):
        hnew.view(flow=True)[i] = h[{oldname: indices}][{oldname: sum}].view(flow=True)

    return hnew

grouping = {
    'TTbar' : ['TTto2L2Nu', 'TTtoLNu2Q', 'TTto4Q'],
    'QCD' : ['QCD_PT-120to170', 'QCD_PT-170to300', 'QCD_PT-1000to1400', 'QCD_PT-1400to1800', 'QCD_PT-1800to2400', 'QCD_PT-300to470', 'QCD_PT-2400to3200', 'QCD_PT-800to1000', 'QCD_PT-600to800', 'QCD_PT-470to600'],
    'ZJets' : ['Zto2Q-4Jets_HT-200to400', 'Zto2Q-4Jets_HT-800', 'Zto2Q-4Jets_HT-600to800', 'Zto2Q-4Jets_HT-400to600'],
    'W' : ['Wto2Q-3Jets_HT-200to400', 'Wto2Q-3Jets_HT-800', 'Wto2Q-3Jets_HT-600to800', 'Wto2Q-3Jets_HT-400to600'],
    'VV' : ['WWto2L2Nu', 'WWto4Q', 'ZZto2L2Nu', 'ZZto2Nu2Q', 'ZZto2L2Q', 'WWtoLNu2Q', 'WZtoLNu2Q', 'WZto3LNu', 'WZto2L2Q'],
    'Bkg. H' : ['ggZH_Hto2B_Zto2L_M-125', 'ZH_Hto2B_Zto2L_M-125', 'ZH_Hto2B_Zto2Q_M-125', 'WplusH_Hto2B_WtoLNu_M-125', 'WminusH_Hto2B_WtoLNu_M-125', 'WplusH_Hto2B_Wto2Q_M-125', 'WminusH_Hto2B_Wto2Q_M-125', 'VBFHto2B_M-125_dipoleRecoilOn', 'ggZH_Hto2B_Zto2Q_M-125', 'ttHto2B_M-125', 'ggZH_Hto2B_Zto2Nu_M-125'],
    'ggF' : ['GluGluHto2B_PT-200_M-125'],
    'VBF' : ['VBFHto2B_M-125_dipoleRecoilOn'],
    'WH' : ['WplusH_Hto2B_WtoLNu_M-125', 'WminusH_Hto2B_WtoLNu_M-125', 'WplusH_Hto2B_Wto2Q_M-125', 'WminusH_Hto2B_Wto2Q_M-125'],
    'ZH' : ['ggZH_Hto2B_Zto2L_M-125', 'ggZH_Hto2B_Zto2Q_M-125', 'ggZH_Hto2B_Zto2Nu_M-125', 'ZH_Hto2B_Zto2L_M-125', 'ZH_Hto2B_Zto2Q_M-125'],
    'ttH' : ['ttHto2B_M-125'],
}

output = group(outsum["templates"], "dataset", "process", grouping)

del outsum

picklename = f"{out_dir}/mc.pkl"
if os.path.isfile(picklename):
    os.remove(picklename)

outfile = open(picklename, 'wb')
pickle.dump({
        "hist" : output,
        "lumi" : total_lumi,
    }, outfile, protocol=-1)
outfile.close()

Loading ../outfiles/Run3Summer22EE/hist/dask_QCD.coffea
Loading ../outfiles/Run3Summer22EE/hist/dask_Zto2Q-4Jets.coffea
Loading ../outfiles/Run3Summer22EE/hist/dask_Wto2Q-3Jets.coffea
Loading ../outfiles/Run3Summer22EE/hist/dask_TT.coffea
Loading ../outfiles/Run3Summer22EE/hist/dask_VV.coffea
Loading ../outfiles/Run3Summer22EE/hist/dask_Hto2B.coffea


/cvmfs/sft.cern.ch/lcg/views/LCG_104/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/hist/basehist.py:99: UserWarning: Please use 'Weight()' instead of 'Weight'
  warnings.warn(msg)
/cvmfs/sft.cern.ch/lcg/views/LCG_104/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/hist/basehist.py:325: UserWarning: List indexing selection is experimental. Removed bins are not placed in overflow.
  return super().__getitem__(self._index_transform(index))


# Grouping collision data together <a class="anchor" id="grouping"></a>
[Back to Table of Contents](#toc)

In [4]:
# Insert directory from which the data samples are read. Here I am reading the samples
# from the "cutflow" step.

step = "cutflow"
out_dir = f"../outfiles/Run3Summer22EE/{step}/data"

In [5]:
from collections import defaultdict 
import os
from coffea.lumi_tools import LumiData, LumiList

path_lumi_csv = "../data/lumi/Run3_2022_2023_Golden2.csv"
lumidata = LumiData(path_lumi_csv)

outsum = defaultdict()
total_lumi = 0

started = 0
with open("../data/inputfiles/Run3Summer22EE/data/files.txt") as f:
    filelist = [line.rstrip() for line in f]
    
for file in filelist:
    filename = f"{out_dir}/{file}"

    if os.path.isfile(filename):
        out = util.load(filename)[0]
        
        for dataset, lumilist in out['lumilist'].items():
            lumi_list = out['lumilist'][dataset]
            lumi_list.unique()
            lumi = lumidata.get_lumi(lumi_list)
            total_lumi += lumi
            
        if started == 0:
            outsum['templates'] = out['hist']
            outsum['sumw'] = out['sumw']
            started += 1
        else:
            outsum['templates'] += out['hist']
            for k,v in out['sumw'].items():
                outsum['sumw'][k] = v
        del out

print()
print(f"Total lumi: {total_lumi:.2f} /pb")
print(f"Total lumi: {total_lumi/1000:.2f} /fb")
        
def group(h: hist.Hist, oldname: str, newname: str, grouping: dict):
    hnew = hist.Hist(
        hist.axis.StrCategory(grouping, name=newname),
        *(ax for ax in h.axes if ax.name != oldname),
        storage=h._storage_type,
    )
    for i, indices in enumerate(grouping.values()):
        hnew.view(flow=True)[i] = h[{oldname: sum}].view(flow=True)

    return hnew
        
grouping = {
    'Run3Summer22EE': []
}

output = group(outsum["templates"], "dataset", "process", grouping)

del outsum

picklename = f"{out_dir}/data.pkl"
if os.path.isfile(picklename):
    os.remove(picklename)

outfile = open(picklename, 'wb')
pickle.dump(output, outfile, protocol=-1)
outfile.close()


Total lumi: 19647.58 /pb
Total lumi: 19.65 /fb


/cvmfs/sft.cern.ch/lcg/views/LCG_104/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/hist/basehist.py:99: UserWarning: Please use 'Weight()' instead of 'Weight'
  warnings.warn(msg)
